In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
dataset_ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")

In [3]:
len(dataset_ftse_mib)

1268

In [61]:
stock_symbols_ftse_mib = ["A2A.MI", "AMP.MI", "ATL.MI", "AZM.MI", "BAMI.MI", "BGN.MI", "BMED.MI", "BPE.MI", "BZU.MI", "CNHI.MI", "CPR.MI", "DIA.MI", "ENEL.MI", "ENI.MI", "EXO.MI", "FBK.MI", "G.MI", "HER.MI",
                          "IG.MI", "INW.MI", "IP.MI", "ISP.MI", "LDO.MI", "MB.MI", "MONC.MI", "NEXI.MI", "PIRC.MI", "PRY.MI", "PST.MI", "RACE.MI", "REC.MI", "SPM.MI", "SRG.MI", "STLA.MI", "STM.MI", "TEN.MI", "TIT.MI", "TRN.MI", "UCG.MI", "UNI.MI", "FTSEMIB.MI"]


In [63]:
dataframes = {}
for sym in stock_symbols_ftse_mib:
  frame = pd.read_csv("./Datasets/"+sym+".csv")
  dataframes[sym] = frame

In [4]:
def get_stocks_by_week(df, week):
    """
    Returns a dataframe of stocks for a given week
    """
    return df[df['Week Number'] == week]

In [118]:
def get_portfolio_weekly_return(portfolio, weekNumber):
  totalReturn = 0
  
  for index, row in portfolio.iterrows():
    
    stock_dataframe = pd.read_csv("./Datasets/"+row["Stock Symbol"]+".csv")
    stock_log_ret_next_day = stock_dataframe.iloc[180+int(weekNumber)+8]["log_ret"] if 180+int(weekNumber)+8 < len(stock_dataframe) else 0
    
    totalReturn += stock_log_ret_next_day / len(portfolio)
  return totalReturn


In [109]:
def get_ftse_mib_weekly_return(week_number):
  ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")
  prev_week_ret = ftse_mib.iloc[180+week_number-7]["log_ret"]
  next_week_ret = ftse_mib.iloc[180+week_number]["log_ret"]
  return next_week_ret-prev_week_ret


In [115]:
def get_ftse_mib_risk(week_number): 
  ftse_mib = pd.read_csv("./Datasets/FTSEMIB.MI.csv")
  variance = ftse_mib.iloc[180+week_number-7:180+week_number]["log_ret"].var()
  return math.sqrt(variance)

In [7]:
test = pd.read_csv("./results.csv")

In [8]:
test.head()

,Unnamed: 0,Stock Symbol,Week Number,Alpha,Beta,R-Squared,Risk,Res Std. Error,Stock Weekly Return
0,0,A2A.MI,0,-0.000713,0.779469,0.293579,0.000174,0.007567,-0.018795
1,1,A2A.MI,7,-0.000081,-0.035455,0.000753,0.000279,0.000026,0.003708
2,2,A2A.MI,14,-0.000423,-0.039381,0.000963,0.000132,0.000034,0.019859
3,3,A2A.MI,21,-0.000870,-0.048152,0.001466,0.000078,0.000052,-0.027421
4,4,A2A.MI,28,-0.001095,-0.067737,0.003573,0.000200,0.000146,0.020515


In [9]:
def get_sorted_df(dataframe, sort_method):
  dataframe = dataframe.sort_values(by=sort_method, ascending=False)
  return dataframe


In [111]:
def get_portfolio_risk(portfolio, weekNumber):
  risk = 0
  columns = portfolio["Stock Symbol"].unique()
  filtered_portfolio = pd.DataFrame(columns=columns)
  
  for index, row in portfolio.iterrows():
    filtered_portfolio[row["Stock Symbol"]] = dataframes[row["Stock Symbol"]].iloc[180+weekNumber:180+weekNumber+7]["log_ret"]
  
  portfolio_variance = 0
  cov_matrix = filtered_portfolio.cov()
  for index, row in portfolio.iterrows():
    for index2, row2 in portfolio.iterrows():
      covariance = cov_matrix.loc[row["Stock Symbol"], row2["Stock Symbol"]]
      portfolio_variance+= 2* (1/len(portfolio)) * covariance
  return math.sqrt(portfolio_variance)


Test: take top 15 and bottom 15 based on beta

In [122]:
final_dataframe = pd.DataFrame(columns=[
                               "Week", "portfolio Weekly Returns Beta", "portfolio Weekly Returns R-Squared", "portfolio Weekly Returns Risk", "portfolio Weekly Returns Res Std Error", "portfolio Weekly Returns Returns", "Ftse Mib weekly returns","Beta Risk","R-Squared Risk","Variance risk", "Residual-Variance Risk", "Return Risk","FTSE_MIB Risk"])
results = pd.read_csv("./results.csv")
for weekNumber in range(0,len(dataset_ftse_mib)-1,7):
  res = get_stocks_by_week(test, weekNumber)
  res = res[res["Stock Symbol"] != "FTSEMIB.MI"]

  sorted_df_beta = get_sorted_df(res, "Beta")
  sorted_df_rsquared = get_sorted_df(res, "R-Squared")
  sorted_df_risk = get_sorted_df(res, "Risk")
  sorted_df_res_std_error = get_sorted_df(res, "Res Std. Error")
  sorted_df_ret = get_sorted_df(res, "Stock Weekly Return")

  frames_beta = [sorted_df_beta.head(4),  sorted_df_beta.tail(4)]
  frames_rsquared = [sorted_df_rsquared.head(4),  sorted_df_rsquared.tail(4)]
  frames_risk = [sorted_df_risk.head(4),  sorted_df_risk.tail(4)]
  frames_res_std_error = [sorted_df_res_std_error.head(
      4),  sorted_df_res_std_error.tail(4)]
  frames_ret = sorted_df_ret.head(
      8)

  top_bottom_10_beta = pd.concat(frames_beta)
  top_bottom_10_rsquared = pd.concat(frames_rsquared)
  top_bottom_10_risk = pd.concat(frames_risk)
  top_bottom_10_res_std_error = pd.concat(frames_res_std_error)
  top_bottom_10_ret = frames_ret

  risk_beta = get_portfolio_risk(top_bottom_10_beta,weekNumber)
  risk_rsquared = get_portfolio_risk(top_bottom_10_rsquared, weekNumber)
  risk_variance = get_portfolio_risk(top_bottom_10_risk, weekNumber)
  risk_errors = get_portfolio_risk(top_bottom_10_res_std_error, weekNumber)
  risk_returns = get_portfolio_risk(top_bottom_10_ret, weekNumber)
  risk_ftse_mib = get_ftse_mib_risk(weekNumber)
  if len(top_bottom_10_beta) > 0:
    portfolio_return_beta = get_portfolio_weekly_return(
        top_bottom_10_beta, weekNumber)
    portfolio_return_rsquared = get_portfolio_weekly_return(
        top_bottom_10_rsquared, weekNumber)
    portfolio_return_risk = get_portfolio_weekly_return(
        top_bottom_10_risk, weekNumber)
    portfolio_return_res_std_error = get_portfolio_weekly_return(
        top_bottom_10_rsquared, weekNumber)
    portfolio_return_ret = get_portfolio_weekly_return(
        top_bottom_10_ret, weekNumber)
    ftse_mib_return = get_ftse_mib_weekly_return(weekNumber)
    final_dataframe.loc[len(final_dataframe)] = [
        weekNumber, portfolio_return_beta, portfolio_return_rsquared,portfolio_return_risk,portfolio_return_res_std_error,portfolio_return_ret, ftse_mib_return, risk_beta, risk_rsquared, risk_variance, risk_errors, risk_returns,risk_ftse_mib]


In [123]:
final_dataframe.drop(final_dataframe.tail(1).index, inplace=True)

In [125]:
final_dataframe.head()

,Week,portfolio Weekly Returns Beta,portfolio Weekly Returns R-Squared,portfolio Weekly Returns Risk,portfolio Weekly Returns Res Std Error,portfolio Weekly Returns Returns,Ftse Mib weekly returns,Beta Risk,R-Squared Risk,Variance risk,Residual-Variance Risk,Return Risk,FTSE_MIB Risk
0,0.0,0.003356,0.017885,-0.016740,0.017885,-0.011514,-0.008566,0.037526,0.022640,0.038310,0.037526,0.046481,0.011579
1,7.0,-0.003819,-0.008952,-0.008847,-0.008952,-0.005936,-0.023240,0.028898,0.041147,0.022614,0.024210,0.018504,0.006071
2,14.0,-0.007772,-0.006352,-0.001215,-0.006352,-0.001089,0.020911,0.030402,0.036743,0.045071,0.031556,0.057847,0.013868
3,21.0,0.021665,0.022250,0.021624,0.022250,0.021625,-0.015410,0.030618,0.027949,0.028296,0.022042,0.025147,0.008636
4,28.0,-0.003145,0.002388,-0.001930,0.002388,-0.005617,0.015685,0.021292,0.023943,0.037571,0.024089,0.037612,0.010359


In [126]:
import plotly.express as px


In [127]:
final_dataframe.head(1)

,Week,portfolio Weekly Returns Beta,portfolio Weekly Returns R-Squared,portfolio Weekly Returns Risk,portfolio Weekly Returns Res Std Error,portfolio Weekly Returns Returns,Ftse Mib weekly returns,Beta Risk,R-Squared Risk,Variance risk,Residual-Variance Risk,Return Risk,FTSE_MIB Risk
0,0.0,0.003356,0.017885,-0.01674,0.017885,-0.011514,-0.008566,0.037526,0.02264,0.03831,0.037526,0.046481,0.011579


In [128]:
initial_investment = 100
portfolio_beta_total_return = initial_investment
portfolio_rsquared_total_return = initial_investment
portfolio_risk_total_return = initial_investment
portfolio_res_std_err_total_return = initial_investment
portfolio_returns_total_return = initial_investment
index_total_return = initial_investment
first = True
final_returns = pd.DataFrame(
    columns=["Week", "Beta Portfolio", "R-Squared Portfolio","Variance Portfolio","Residual variance Portfolio","Return Portfolio", "FTSE_MIB", "Beta Risk","R-Squared Risk", "Variance Risk","Error Risk","Returns risk"])
for index, row in final_dataframe.iterrows():
  
  portfolio_beta_total_return *= math.exp(row["portfolio Weekly Returns Beta"])
  portfolio_rsquared_total_return *= math.exp(
      row["portfolio Weekly Returns R-Squared"])
  portfolio_risk_total_return *= math.exp(
      row["portfolio Weekly Returns Risk"])
  portfolio_res_std_err_total_return *= math.exp(
      row["portfolio Weekly Returns Res Std Error"])
  portfolio_returns_total_return *= math.exp(
      row["portfolio Weekly Returns Returns"])
  index_total_return *= math.exp(row["Ftse Mib weekly returns"])
 
  final_returns.loc[len(final_returns)] = [row["Week"],
                                           portfolio_beta_total_return, 
                                           portfolio_rsquared_total_return, 
                                           portfolio_risk_total_return, 
                                           portfolio_res_std_err_total_return, 
                                           portfolio_returns_total_return, 
                                           index_total_return,
                                            row["Beta Risk"].mean(), 
                                            row["R-Squared Risk"].mean(), 
                                            row["Variance risk"].mean(), 
                                            row["Residual-Variance Risk"].mean(), 
                                            row["Return Risk"].mean()]
  

In [129]:
final_returns.head(20)

,Week,Beta Portfolio,R-Squared Portfolio,Variance Portfolio,Residual variance Portfolio,Return Portfolio,FTSE_MIB,Beta Risk,R-Squared Risk,Variance Risk,Error Risk,Returns risk
0,0.0,100.336123,101.804558,98.339917,101.804558,98.855226,99.147019,0.037526,0.022640,0.038310,0.037526,0.046481
1,7.0,99.953690,100.897315,97.473737,100.897315,98.270114,96.869363,0.028898,0.041147,0.022614,0.024210,0.018504
2,14.0,99.179879,100.258434,97.355394,100.258434,98.163141,98.916344,0.030402,0.036743,0.045071,0.031556,0.057847
3,21.0,101.352097,102.514169,99.483562,102.514169,100.309040,97.403750,0.030618,0.027949,0.028296,0.022042,0.025147
4,28.0,101.033801,102.759311,99.291740,102.759311,99.747227,98.943551,0.021292,0.023943,0.037571,0.024089,0.037612
5,35.0,98.782806,102.132646,96.912003,102.132646,98.633104,99.345388,0.022053,0.027643,0.027578,0.021212,0.027189
6,42.0,99.229144,102.640656,95.929161,102.640656,100.112209,99.454101,0.044777,0.038782,0.035983,0.039251,0.053228
7,49.0,98.267560,100.424243,94.419214,100.424243,99.508046,100.356524,0.013269,0.014155,0.019375,0.017323,0.015441
8,56.0,97.746660,100.101062,93.627065,100.101062,98.543902,99.550320,0.042261,0.023480,0.028560,0.042261,0.034034
9,63.0,97.257082,99.054791,93.319529,99.054791,99.530921,98.532928,0.023445,0.026475,0.038704,0.030062,0.017112


In [130]:
len(final_returns)

155

## Calculation of portfolio risks and expected returns

Portfolios risks

In [136]:
print("R-Squared portfolio risk: " +
      str(final_returns["R-Squared Risk"].mean()*math.sqrt(52)))
print("Beta portfolio risk: " +
      str(final_returns["Beta Risk"].mean()*math.sqrt(52)))
print("Variance portfolio risk: " +
      str(final_returns["Variance Risk"].mean()*math.sqrt(52)))
print("Res std error portfolio risk: " +
      str(final_returns["Error Risk"].mean()*math.sqrt(52)))
print("Returns portfolio risk: " +
      str(final_returns["Returns risk"].mean()*math.sqrt(52)))
print("FTSE-MIB portfolio risk: " +
      str(final_dataframe["FTSE_MIB Risk"].mean()*math.sqrt(52)))



R-Squared portfolio risk: 0.31235303050335256
Beta portfolio risk: 0.3407237785485088
Variance portfolio risk: 0.3247240611043499
Res std error portfolio risk: 0.33707970993026176
Returns portfolio risk: 0.3427175173264121
FTSE-MIB portfolio risk: 0.08630134543054792


### Portfolios expected returns

In [132]:
print("R-Squared portfolio expected return: " +
      str(final_dataframe["portfolio Weekly Returns R-Squared"].mean()*52))
print("Beta portfolio expected return: " +
      str(final_dataframe["portfolio Weekly Returns Beta"].mean()*52))
print("Variance portfolio expected return: " +
      str(final_dataframe["portfolio Weekly Returns Risk"].mean()*52))
print("Res std error portfolio expected return: " +
      str(final_dataframe["portfolio Weekly Returns Res Std Error"].mean()*52))
print("Returns portfolio expected return: " +
      str(final_dataframe["portfolio Weekly Returns Returns"].mean()*52))
print("FTSE-MIB portfolio expected return: " +
      str(final_dataframe["Ftse Mib weekly returns"].mean()*52))


R-Squared portfolio expected return: 0.012004322118663565
Beta portfolio expected return: -0.028597152708948883
Variance portfolio expected return: -0.03390959075942273
Res std error portfolio expected return: 0.012004322118663565
Returns portfolio expected return: -0.018981272329831524
FTSE-MIB portfolio expected return: 0.0026513448671930486


### Ratios

In [137]:
print("Beta ratio: " +
      str((final_dataframe["portfolio Weekly Returns Beta"].mean()*52)/(final_returns["R-Squared Risk"].mean()*math.sqrt(52))))
print("R-Squared ratio: " +
      str((final_dataframe["portfolio Weekly Returns R-Squared"].mean()*100*52)/(final_returns["Beta Risk"].mean()*100*math.sqrt(52))))
print("Variance ratio: " +
      str((final_dataframe["portfolio Weekly Returns Risk"].mean()*100*52)/(final_returns["Variance Risk"].mean()*100*math.sqrt(52))))
print("Error variance ratio: " +
      str((final_dataframe["portfolio Weekly Returns Res Std Error"].mean()*100*52)/(final_returns["Error Risk"].mean()*100*math.sqrt(52))))
print("Returns ratio: " +
      str((final_dataframe["portfolio Weekly Returns Returns"].mean()*100*52)/(final_returns["Returns risk"].mean()*100*math.sqrt(52))))
print("FTSE MIB ratio: " +
      str((final_dataframe["Ftse Mib weekly returns"].mean()*100*52)/(final_dataframe["FTSE_MIB Risk"].mean()*100*math.sqrt(52))))


Beta ratio: -0.09155394670852071
R-Squared ratio: 0.03523182963573089
Variance ratio: -0.10442586436034347
Error variance ratio: 0.03561271048069648
Returns ratio: -0.05538459918216937
FTSE MIB ratio: 0.03072194128568657


### Portfolios Rations

In [134]:
# first = True
# final_returns = pd.DataFrame(columns=["Week","Portfolio_Ret", "FTSE_MIB_Ret"])
# for index, row in final_dataframe.iterrows():
  
  
#   p_perc_return = (math.exp(row["portfolio Weekly Returns"]))
#   i_perc_return = (math.exp(row["Ftse Mib weekly returns"]))
#   if row["portfolio Weekly Returns"] < 0:
#     p_perc_return *=-1
#   if row["Ftse Mib weekly returns"]< 0:
#     i_perc_return *=-1
#   portfolio_total_return = initial_investment * \
#       p_perc_return if first else portfolio_total_return*p_perc_return
#   index_total_return = initial_investment * \
#       i_perc_return if first else index_total_return*i_perc_return
#   first = False
#   final_returns.loc[len(final_returns)] = [row["Week"],portfolio_total_return, index_total_return]

## Returns charts

In [135]:
fig = px.line(final_returns, x="Week", y=["Beta Portfolio", "R-Squared Portfolio", "Variance Portfolio", "Residual variance Portfolio", "Return Portfolio", "FTSE_MIB"],
              title='Weekly returns, for an investment of 100€', labels=dict(value="Returns(€)", variable="Portfolio"), width=1100 )
fig.show()
